## Programming Exercise 3: Multi-class Classification and Neural Networks

#### Setup

In [64]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt

data = loadmat('ex3data1.mat')
X = data['X']
y = data['y']

weights = loadmat('ex3weights.mat')
theta1 = weights['Theta1']
theta2 = weights['Theta2']
m = X.shape[0]
X_pad = np.c_[np.ones((m, 1)), X]

#### Displaying Data

In [89]:
rand_indices = np.random.permutation(m)
# the magic number of a 20 appears since 
sel1 = X_pad[rand_indices[0:20], 1:].reshape(-1,20).T
sel2 = X_pad[rand_indices[0:20], 1:].reshape(-1,20).T

print sel1.shape
# plt.imshow(sel)
# plt.axis('off');
# plt.show()

(20, 400)
